In [9]:
import cv2
import pandas as pd
from ultralytics import YOLO
from moviepy import VideoFileClip, concatenate_videoclips
import numpy as np
import os

# --- KONFIGŪRACIJA ---
VIDEO_PATH = "data/Lakers_vs_Thunder.mp4"
OUTPUT_FOLDER = "results"
OUTPUT_PATH = os.path.join(OUTPUT_FOLDER, "summary.mp4")
CSV_PATH = os.path.join(OUTPUT_FOLDER, "analysis_data.csv")
MODEL_NAME = "models/yolo11s.pt"
TARGET_DURATION = 120
SCAN_INTERVAL = 10

In [10]:
def analyze_video(video_path, model):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"KLAIDA: Video failas nerastas! Tikrinta vieta: {os.path.abspath(video_path)}")

    print(f"Pradedama video analizė: {video_path}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError("KLAIDA: OpenCV negali atidaryti video failo. Failas gali būti sugadintas arba netinkamo formato.")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    yolo = YOLO(model)
    data = []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % SCAN_INTERVAL == 0:
            results = yolo(frame, verbose=False, classes=[0, 32])

            person_count = 0
            ball_detected = 0

            for r in results:
                boxes = r.boxes
                for box in boxes:
                    cls = int(box.cls[0])
                    if cls == 0:
                        person_count += 1
                    elif cls == 32:
                        ball_detected = 1

            score = (person_count * 1) + (ball_detected * 50)

            data.append({
                'frame': frame_idx,
                'time_sec': frame_idx / fps,
                'score': score
            })

            if len(data) % 10 == 0:
                print(f"Progresas: {frame_idx}/{total_frames} | Balas: {score}...", end='\r')

        frame_idx += 1

    cap.release()
    df = pd.DataFrame(data)
    print("\nAnalizė baigta.")
    return df, fps


In [11]:
def create_summary(video_path, df, fps, target_duration):
    if df.empty:
        print("KLAIDA: Jokie duomenys nebuvo surinkti.")
        return

    print("Generuojamas sklandus montažas...")

    # Galime imti viršutinį 15-20% geriausių momentų
    threshold = df['score'].quantile(0.85)
    important_frames = df[df['score'] >= threshold].copy()

    if important_frames.empty:
        print("Nerasta pakankamai svarbių momentų.")
        return

    # 2. Sukuriame intervalus su BUFERIU (pvz., +/- 2 sekundės)
    BUFFER_BEFORE = 2.0  # s.
    BUFFER_AFTER = 1.5   # s.

    intervals = []
    for _, row in important_frames.iterrows():
        start = max(0, row['time_sec'] - BUFFER_BEFORE)
        end = row['time_sec'] + BUFFER_AFTER
        intervals.append([start, end])

    intervals.sort()
    merged_intervals = []
    if intervals:
        curr_start, curr_end = intervals[0]
        for next_start, next_end in intervals[1:]:
            if next_start <= curr_end + 1.0:
                curr_end = max(curr_end, next_end)
            else:
                merged_intervals.append((curr_start, curr_end))
                curr_start, curr_end = next_start, next_end
        merged_intervals.append((curr_start, curr_end))

    original_clip = VideoFileClip(video_path)
    subclips = []

    print(f"Po suliejimo rasta epizodų: {len(merged_intervals)}")

    for start, end in merged_intervals:
        end_time = min(original_clip.duration, end)
        if start < end_time:
            subclip = original_clip.subclipped(start, end_time)
            subclips.append(subclip)

    if subclips:
        final_clip = concatenate_videoclips(subclips)
        if final_clip.duration > target_duration:
            final_clip = final_clip.subclipped(0, target_duration)

        final_clip.write_videofile(OUTPUT_PATH, codec="libx264", audio_codec="aac")
        print(f"Eksportuota! Trukmė: {final_clip.duration:.2f}s")
    else:
        print("Nepavyko sukurti klipų.")

In [12]:
if __name__ == "__main__":
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print(f"Sukurtas aplankas: {OUTPUT_FOLDER}")

    try:
        df_scores, fps_rate = analyze_video(VIDEO_PATH, MODEL_NAME)

        df_scores.to_csv(CSV_PATH, index=False)
        print(f"Duomenys išsaugoti: {CSV_PATH}")

        create_summary(VIDEO_PATH, df_scores, fps_rate, TARGET_DURATION)

    except FileNotFoundError as e:
        print("\n!!! SUSTOTA DĖL KLAIDOS !!!")
        print(e)
        print("PATARIMAS: Patikrinkite, ar failo pavadinimas 'VIDEO_PATH' kintamajame sutampa su failu 'data' aplanke.")
    except Exception as e:
        print(f"Įvyko nenumatyta klaida: {e}")

Pradedama video analizė: data/Lakers_vs_Thunder.mp4
Progresas: 35490/35519 | Balas: 50...
Analizė baigta.
Duomenys išsaugoti: results/analysis_data.csv
Generuojamas sklandus montažas...
Po suliejimo rasta epizodų: 16
MoviePy - Building video results/summary.mp4.
MoviePy - Writing audio in summaryTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video results/summary.mp4



MoviePy - Done !
MoviePy - video ready results/summary.mp4
Eksportuota! Trukmė: 120.00s
